# Confidence Calibration

In our scoring application, we use positive confidence (possibility) as score of the sample. Since deep models trend to be overconfident, we cannot get a reasonable score directly with positive possibility.  
Following [On Calibration of Modern Neural Networks](https://arxiv.org/pdf/1706.04599.pdf), we calibrate network confidence by minimizing NLL of correct class with respect to a temperature parameter.  
Implementation with [temperature_scaling](https://github.com/gpleiss/temperature_scaling).  

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import os
import sys

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from sklearn import metrics
from matplotlib import pyplot as plt

from temperature_scaling import ModelWithTemperature

from dataset import *
from resnet import *
from classifier import *

In [2]:
# consts
TRAIN_NAME = 'uda-test'
TRAIN_ID = '10'
EPOCH = 99

# data consts
ROOT_PATH = '/home/xd/data/chromo/class-2/uda'
NUM_CLASSES = 2 # fg + 1(bg)
INPUT_SIZE = 512
BATCH_SIZE = 1
NUM_WORKERS = 4

# trainer consts
DEVICE = 'cuda:0'


In [3]:
# val_loader
val_trans = transforms.Compose([
    transforms.ToPILImage(),
    PadSquare(),
    transforms.Resize((INPUT_SIZE, INPUT_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

val_dataset = UdaDataset(
    ROOT_PATH,
    training=False,
    image_ext='.png',
    sup_transform=val_trans
)

val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    # batch_sampler=val_sampler,
    shuffle=False,
    num_workers=NUM_WORKERS,
    pin_memory=True
)

FileNotFoundError: [Errno 2] No such file or directory: '/home/xd/data/chromo/class-2/uda/labeled/0'

In [ ]:
# load trained model
device = torch.device(DEVICE)

model = resnet50(pretrained=True, num_classes=NUM_CLASSES)

checkpoint_path = os.path.join('./models', '{}_{}'.format(TRAIN_NAME, TRAIN_ID), '{:0>3d}.pth'.format(EPOCH))
cp_state_dict = torch.load(checkpoint_path, map_location='cpu')

if 'module' in list(cp_state_dict.keys())[0]:
    new_state_dict = {}
    
    for key, value in cp_state_dict.items():
        new_state_dict[key.split('.', 1)[1]] = value
    
    model.load_state_dict(new_state_dict)
else:
    model.load_state_dict(cp_state_dict)
    
model = model.to(device)
model.eval()

In [ ]:
# calibration

scaled_model = ModelWithTemperature(model)
scaled_model.set_temperature(val_loader)